# Test Scratch Baord

This is a scratch board for testing the data loaded.

In [1]:
import configparser
import os
import datetime

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import weekofyear, monotonically_increasing_id, udf, col, from_unixtime, year, month, dayofmonth, hour, weekofyear, date_format, to_timestamp, to_date, from_unixtime
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['default']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['default']['AWS_SECRET_ACCESS_KEY']
os.environ['region']=config['default']['region']
os.environ['region_name']=config['default']['region']

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

sc=spark.sparkContext
hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID'])
hadoop_conf.set("fs.s3n.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY'])
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version", "2")

In [4]:
songs_df = spark.read.parquet("output_data/songs.parquet")
#songs_df = spark.read.parquet("s3n://udacityproject4/songs.parquet")
#songs_df.count()
#songs_df.toPandas().describe()
songs_df.toPandas().head(4)

,song_id,title,duration,year,artist_id
0,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,43.36281,2000,ARPBNLO1187FB3D52F
1,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005,ARDNS031187B9924F0
2,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),326.00771,0,ARLTWXK1187FB5A3F8
3,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),267.70240,0,ARKFYS91187B98E58F


In [7]:
artists_df = spark.read.parquet("output_data/artists.parquet")
#songs_df = spark.read.parquet("s3n://udacityproject4/songs.parquet")
artists_df.toPandas().head(4)

,artist_id,name,location,latitude,longitude
0,ARMAC4T1187FB3FA4C,The Dillinger Escape Plan,"Morris Plains, NJ",40.82624,-74.47995
1,ARNF6401187FB57032,Sophie B. Hawkins,"New York, NY [Manhattan]",40.79086,-73.96644
2,AROUOZZ1187B9ABE51,Willie Bobo,"New York, NY [Spanish Harlem]",40.79195,-73.94512
3,ARI2JSK1187FB496EF,Nick Ingman;Gavyn Wright,"London, England",51.50632,-0.12714


In [8]:
users_df = spark.read.parquet("output_data/users.parquet")
#songs_df = spark.read.parquet("s3n://udacityproject4/songs.parquet")
users_df.toPandas().head(3)


,user_id,first_name,last_name,level,ts,gender
0,88,Mohammad,Rodriguez,paid,1543590783796,M
1,60,Devin,Larson,free,1543416119796,M
2,68,Jordan,Rodriguez,free,1543584463796,F


In [9]:
time_df = spark.read.parquet("output_data/time.parquet")
#songs_df = spark.read.parquet("s3n://udacityproject4/songs.parquet")
#time_df.toPandas().head(3)
time_df.dropDuplicates(['start_time']).count()

8023

In [10]:
songplays_df = spark.read.parquet("output_data/songplays.parquet")
#songs_df = spark.read.parquet("s3n://udacityproject4/songs.parquet")
songplays_df.toPandas().head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,songplay_id,year,month
0,Elena,Logged In,Lily,F,5,Koch,269.58322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,...,Dubai UAE,15.47491,Elena,269.58322,1,SOZCTXZ12AB0182364,Setanta matins,0,2018,11
